In [43]:
! pip install pyautogen librosa

  Using cached librosa-0.10.2.post1-py3-none-any.whl.metadata (8.6 kB)
  Using cached audioread-3.0.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached soundfile-0.12.1-py2.py3-none-win_amd64.whl.metadata (14 kB)
  Using cached pooch-1.8.2-py3-none-any.whl.metadata (10 kB)
Using cached librosa-0.10.2.post1-py3-none-any.whl (260 kB)
Using cached audioread-3.0.1-py3-none-any.whl (23 kB)
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ----------------------------------- ---- 2.4/2.7 MB 11.2 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 11.1 MB/s eta 0:00:00
Using cached pooch-1.8.2-py3-none-any.whl (64 kB)
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   --------- ------------------------------ 2.6/11.1 MB 13.7 MB/s eta 0:00:01
   ------------------- -------------------- 5.5/11.1 MB 13.4 MB/s eta 0:00:01
   ----------------------------- ---------- 8.1/11.1 MB 13.2 MB/s eta 0:00:01
   --------------------------------

In [44]:
import os
import yaml

# Load the YAML file
def load_api_key(yml_file):
    with open(yml_file, 'r') as file:
        config = yaml.safe_load(file)  # Safely load the YAML file
        return config.get('openai_key')  # Retrieve the 'api_key'

# Usage
os.environ['OPENAI_API_KEY'] =load_api_key('chatgpt_api_credentials.yml')


In [45]:
import autogen
from autogen import config_list_from_json
from autogen import UserProxyAgent, AssistantAgent, GroupChat, GroupChatManager

In [46]:
configs = [{
    "model": "gpt-4o-mini",
    "api_key": os.environ['OPENAI_API_KEY']
}
]




In [47]:
gpt4o_config ={
    "cache_seed": 42, #change the cache seed for different trials
    "temperature":0,
    "config_list": configs,
    "timeout": 120,
}

user_proxy = autogen.UserProxyAgent(
    name="Admin",
    system_message="A human admin. Interact with a planner to discuss the plan of execution. The plan needs to be approved by this admin.",
    code_execution_config=False
)

planner = autogen.AssistantAgent(
    name="Planner",
    system_message='Planner. Suggest a plan. Revise the plan based on feedback froma critic agent.\
          The plan may involve an engineer who can write code and a scientist who doesnt write code.\
            Explain the plan first, Be clear which step is performed by engineer and which i performed by scientist'
)

In [48]:
engineer = AssistantAgent(
    name="Engineer",
    llm_config=gpt4o_config,
    system_message="""Engineer. You follow an approved plan. You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
""",
)


In [49]:
scientist = AssistantAgent(
    name="Scientist",
    llm_config=gpt4o_config,
    system_message="""Scientist. You follow an approved plan.
    You are able to categorize papers after seeing their abstracts printed.
    You don't write code.""",
)

In [50]:
planner = AssistantAgent(
    name="Planner",
    system_message="""Planner. Suggest a plan. Revise the plan based on feedback
    from admin and critic, until admin approval. The plan may involve an engineer
    who can write code and a scientist who doesn't write code. Explain the plan 
    first. Be clear which step is performed by an engineer, and which step is 
    performed by a scientist.
    """,
    llm_config=gpt4o_config
)


In [59]:
from pathlib import Path
work_dir = Path('video_papers')


In [60]:
executor = UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer\
        and report the result.",
    human_input_mode="NEVER",
    code_execution_config={"last_n_messages": 3,
                           "work_dir": work_dir, "use_docker":False},

)


In [61]:
critic = AssistantAgent(
    name="Critic", 
    system_message="""Critic. Double check plan, claims, code from other
    agents and provide feedback. Check whether the plan includes adding 
    verifiable info such as source URL.
    """,
    llm_config=gpt4o_config,
    
)


In [62]:
groupchat = GroupChat(
    agents=[user_proxy, engineer, scientist, planner, executor, critic], 
    messages=[], max_round=20
)
manager = GroupChatManager(groupchat=groupchat, 
                           llm_config=gpt4o_config)

In [65]:
output_report = user_proxy.initiate_chat(
    manager,
    message="Find 10 state of the art video generation paper architectures (such as VideoGAN, Temporal GAN, Stable diffusion, VideoGPT, Imitation learning Model). Create markdown table summarizing these architectures named video_paper.md as well as markdown for each of the architecture saved as architecture_name.md and python code for each architectures as architecture.py."
)

Admin (to chat_manager):

Find various state of the art video generation paper architectures. Create markdown table summarizing these architectures named video_paper.md as well as markdown for each of the architecture saved as architecture_name.md and python code for each architectures as architecture.py.

--------------------------------------------------------------------------------

Next speaker: Planner

Planner (to chat_manager):

### Plan for Video Generation Paper Architectures

The goal of this plan is to gather state-of-the-art video generation architectures from recent research papers, summarize them in a markdown table, and create individual markdown files and Python code files for each architecture. 

#### Steps Involved:

1. **Research and Data Collection (Scientist)**
   - The scientist will conduct a literature review to identify and collect state-of-the-art video generation architectures from recent research papers. This will involve searching through academic database

In [29]:
from IPython.display import Markdown

Markdown('paper/2024_audio_processing.md')




# Audio Data Preprocessing Steps

| Preprocessing Step     | Description                                                                 |
|-----------------------|-----------------------------------------------------------------------------|
| Normalization         | Adjusts the amplitude of the audio signal to a standard range.             |
| Noise Reduction       | Removes unwanted background noise from the audio signal.                   |
| Feature Extraction     | Extracts relevant features from the audio signal, such as MFCCs.          |
| Resampling            | Changes the sample rate of the audio signal to a different frequency.      |
| Trimming/Silence Removal | Cuts out silent parts of the audio signal to reduce file size and improve processing. |
